In [34]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
import math
import word_dist_script as ws
import pandas as pd

In [35]:
twitEmbs = ws.load_twitter_embs()

loading finished


In [36]:
load_word2vec = False

if load_word2vec:
    word2vec = ws.load_word2vec_embs()


In [37]:
every_emb = ws.get_every_embeddings(word2vec)

clf = KMeans(n_clusters=5, random_state=0)

clf.fit(every_emb)

pred_emb = clf.predict(every_emb)

c:\Users\Nicklas\anaconda3\envs\syp\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
c:\Users\Nicklas\anaconda3\envs\syp\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [38]:
every_dist = ws.get_every_distance(word2vec)

clf = KMeans(n_clusters=5, random_state=0, n_init=1)

clf.fit(every_dist)

pred_dist = clf.predict(every_dist)



c:\Users\Nicklas\anaconda3\envs\syp\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [39]:
sorted_dist = ws.sort_zip_labels(pred_dist)
groups_dist = ws.group_by_cluster(sorted_dist)

sorted_emb = ws.sort_zip_labels(pred_emb)
groups_emb = ws.group_by_cluster(sorted_emb)

In [40]:
for i, group in enumerate(groups_dist):
    print(i, group)

0 ['journal', 'astronomer', 'book', 'magazine', 'researcher', 'scientist', 'theory', 'university', 'writer']
1 ['album', 'band', 'genre', 'artist', 'instrument', 'poem', 'politics', 'politician', 'song']
2 ['country', 'event', 'field', 'location', 'metrics', 'person', 'product', 'java', 'task']
3 ['algorithm', 'chemical', 'enzyme', 'protein']
4 ['award', 'conference', 'discipline', 'election', 'misc', 'Organisation']


In [41]:
for i, group in enumerate(groups_emb):
    print(i, group)

0 ['astronomer', 'researcher', 'scientist']
1 ['algorithm', 'chemical', 'conference', 'country', 'discipline', 'election', 'event', 'field', 'location', 'metrics', 'instrument', 'Organisation', 'person', 'politics', 'politician', 'product', 'task', 'university']
2 ['journal', 'album', 'award', 'band', 'book', 'genre', 'magazine', 'artist', 'poem', 'java', 'song', 'theory', 'writer']
3 ['enzyme', 'protein']
4 ['misc']


In [42]:
for group in groups_dist:
    ## find distance to centroid
    label = ws.find_label_for_cluster(word2vec, group)
    print(label, group)

journal ['journal', 'astronomer', 'book', 'magazine', 'researcher', 'scientist', 'theory', 'university', 'writer']
artist ['album', 'band', 'genre', 'artist', 'instrument', 'poem', 'politics', 'politician', 'song']
location ['country', 'event', 'field', 'location', 'metrics', 'person', 'product', 'java', 'task']
enzyme ['algorithm', 'chemical', 'enzyme', 'protein']
conference ['award', 'conference', 'discipline', 'election', 'misc', 'Organisation']


In [43]:
for group in groups_emb:
    ## find distance to centroid
    label = ws.find_label_for_cluster_emb(word2vec, group)
    print(label, group)

scientist ['astronomer', 'researcher', 'scientist']
country ['algorithm', 'chemical', 'conference', 'country', 'discipline', 'election', 'event', 'field', 'location', 'metrics', 'instrument', 'Organisation', 'person', 'politics', 'politician', 'product', 'task', 'university']
book ['journal', 'album', 'award', 'band', 'book', 'genre', 'magazine', 'artist', 'poem', 'java', 'song', 'theory', 'writer']
enzyme ['enzyme', 'protein']
misc ['misc']


In [44]:
for group in groups_emb:
    # find most similar word
    centroid = ws.find_vector_for_cluster(word2vec, group)
    most_sim = word2vec.most_similar(centroid)[0]
    print(most_sim)

('scientist', 0.9096941351890564)
('By_Jonas_Elmerraji', 0.5770610570907593)
('album', 0.7002332210540771)
('enzyme', 0.9519215226173401)
('misc', 1.0)


In [45]:
entity2cluster = dict()
for i, pred in enumerate(pred_dist):
    entity2cluster[ws.ENTITY_LABELS_SPLIT[i]] = pred


In [46]:
cluster2label = dict()
for i, group in enumerate(groups_dist):
    label = ws.find_label_for_cluster_emb(word2vec, group)
    cluster2label[i] = label

In [47]:
example = ['journal', 'journal','journal']

In [48]:
mapped = map(lambda entity: entity2cluster[entity], example)

In [52]:
df = pd.read_csv('../data/manual_groups.csv')
entities = df["entity_name"]

In [57]:
def find_problematic_words(word2vec_model, entities):
    PROBLEMATIC_WORDS = []
    for entity in entities:
        try:
            word2vec_model[entity]
        except:
            PROBLEMATIC_WORDS.append(entity)
    return PROBLEMATIC_WORDS

In [58]:
find_problematic_words(word2vec_model=word2vec, entities=entities)

['musicalartist',
 'organisation',
 'politicalparty',
 'academicjournal',
 'chemicalcompound',
 'chemicalelement',
 'astronomicalobject',
 'musicgenre',
 'literarygenre',
 'programlang',
 'musicalinstrument']

In [60]:
problem2work = {"musicalartist": "artist", "organisation": "Organisation", "politicalparty": "politics", "academicjournal": "journal", "chemicalcompound": "chemical", "chemicalelement": "chemical", "astronomicalobject": "astronomer", "musicgenre": "genre", "literarygenre": "genre", "programlang": "java", "musicalinstrument": "instrument"  }

In [74]:
def entity2label(entities):
    return map(lambda entity: cluster2label[entity2cluster[entity]], entities)

In [75]:
working = entities.map(lambda entity: problem2work[entity] if entity in problem2work else entity)

KeyError: 'party'